# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about vision APIs for vision-language models and embedding APIs for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30020 --host 0.0.0.0"
)

wait_for_server("http://localhost:30020")

[2025-02-03 23:51:30] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=30020, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=274301095, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable

[2025-02-03 23:51:49 TP0] Init torch distributed begin.


[2025-02-03 23:51:49 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-03 23:51:50 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

 96%|█████████▌| 22/23 [00:05<00:00,  5.32it/s]

100%|██████████| 23/23 [00:05<00:00,  4.37it/s]
[2025-02-03 23:51:59 TP0] Capture cuda graph end. Time elapsed: 5.31 s


[2025-02-03 23:52:00 TP0] max_total_num_tokens=444372, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2025-02-03 23:52:00] INFO:     Started server process [1946765]
[2025-02-03 23:52:00] INFO:     Waiting for application startup.
[2025-02-03 23:52:00] INFO:     Application startup complete.
[2025-02-03 23:52:00] INFO:     Uvicorn running on http://0.0.0.0:30020 (Press CTRL+C to quit)


[2025-02-03 23:52:01] INFO:     127.0.0.1:52954 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-03 23:52:01 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-03 23:52:01] INFO:     127.0.0.1:52976 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-03 23:52:04] INFO:     127.0.0.1:52964 - "POST /generate HTTP/1.1" 200 OK
[2025-02-03 23:52:04] The server is fired up and ready to roll!


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30020/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-02-03 23:52:07 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, cache hit rate: 2.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-03 23:52:07 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 5.53, #queue-req: 0
[2025-02-03 23:52:07] INFO:     127.0.0.1:52990 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-02-03 23:52:07 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, cache hit rate: 20.63%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-03 23:52:07 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.


[2025-02-03 23:52:07 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.00, gen throughput (token/s): 128.61, #queue-req: 0


[2025-02-03 23:52:07 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.00, gen throughput (token/s): 145.19, #queue-req: 0


[2025-02-03 23:52:08 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.00, gen throughput (token/s): 144.61, #queue-req: 0
[2025-02-03 23:52:08] INFO:     127.0.0.1:52990 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-02-03 23:52:08] INFO:     127.0.0.1:52990 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-02-03 23:52:08 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, cache hit rate: 33.73%, token usage: 0.00, #running-req: 0, #queue-req: 0
It looks like we have a test going

 on. What would you like to test?

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-02-03 23:52:08 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 32.57%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-03 23:52:08 TP0] Decode batch. #running-req: 1, #token: 13, token usage: 0.00, gen throughput (token/s): 131.29, #queue-req: 0


[2025-02-03 23:52:08 TP0] Decode batch. #running-req: 1, #token: 53, token usage: 0.00, gen throughput (token/s): 150.95, #queue-req: 0
[2025-02-03 23:52:08] INFO:     127.0.0.1:52990 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-02-03 23:52:08 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 31.35%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-03 23:52:08 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.


[2025-02-03 23:52:08 TP0] Decode batch. #running-req: 1, #token: 30, token usage: 0.00, gen throughput (token/s): 140.75, #queue-req: 0


[2025-02-03 23:52:09 TP0] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 149.44, #queue-req: 0


[2025-02-03 23:52:09 TP0] Decode batch. #running-req: 1, #token: 110, token usage: 0.00, gen throughput (token/s): 147.86, #queue-req: 0


[2025-02-03 23:52:09 TP0] Decode batch. #running-req: 1, #token: 150, token usage: 0.00, gen throughput (token/s): 146.37, #queue-req: 0
[2025-02-03 23:52:09] INFO:     127.0.0.1:52990 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30020/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-02-03 23:52:09] INFO:     127.0.0.1:53006 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-03 23:52:09] INFO:     127.0.0.1:53006 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-03 23:52:09 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, cache hit rate: 44.53%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-02-03 23:52:10 TP0] Decode batch. #running-req: 1, #token: 69, token usage: 0.00, gen throughput (token/s): 106.92, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-02-03 23:52:12] INFO:     127.0.0.1:53006 - "GET /v1/batches/batch_877219c2-93e0-43e3-8d62-17d11b05a1a9 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-02-03 23:52:12] INFO:     127.0.0.1:53006 - "GET /v1/files/backend_result_file-e835be9d-b442-4104-82ee-b6130b7d8269/content HTTP/1.1" 200 OK


[2025-02-03 23:52:12] INFO:     127.0.0.1:53006 - "DELETE /v1/files/backend_result_file-e835be9d-b442-4104-82ee-b6130b7d8269 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30020/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-02-03 23:52:13] INFO:     127.0.0.1:36128 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-03 23:52:13] INFO:     127.0.0.1:36128 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-03 23:52:13 TP0] Prefill batch. #new-seq: 3, #new-token: 90, #cached-token: 75, cache hit rate: 44.88%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-03 23:52:13 TP0] Prefill batch. #new-seq: 88, #new-token: 2640, #cached-token: 2200, cache hit rate: 45.41%, token usage: 0.00, #running-req: 3, #queue-req: 0
[2025-02-03 23:52:13 TP0] Prefill batch. #new-seq: 9, #new-token: 270, #cached-token: 225, cache hit rate: 45.41%, token usage: 0.01, #running-req: 91, #queue-req: 0


[2025-02-03 23:52:13 TP0] Decode batch. #running-req: 100, #token: 5025, token usage: 0.01, gen throughput (token/s): 630.52, #queue-req: 0


[2025-02-03 23:52:13 TP0] Decode batch. #running-req: 100, #token: 9025, token usage: 0.02, gen throughput (token/s): 12008.36, #queue-req: 0


[2025-02-03 23:52:14 TP0] Decode batch. #running-req: 100, #token: 13025, token usage: 0.03, gen throughput (token/s): 11726.96, #queue-req: 0


...

[2025-02-03 23:52:23] INFO:     127.0.0.1:60460 - "GET /v1/batches/batch_6ea3847b-0585-43c0-9689-918d656a2bf4 HTTP/1.1" 200 OK


[2025-02-03 23:52:26] INFO:     127.0.0.1:60460 - "GET /v1/batches/batch_6ea3847b-0585-43c0-9689-918d656a2bf4 HTTP/1.1" 200 OK


[2025-02-03 23:52:29] INFO:     127.0.0.1:60460 - "GET /v1/batches/batch_6ea3847b-0585-43c0-9689-918d656a2bf4 HTTP/1.1" 200 OK


[2025-02-03 23:52:32] INFO:     127.0.0.1:60460 - "GET /v1/batches/batch_6ea3847b-0585-43c0-9689-918d656a2bf4 HTTP/1.1" 200 OK


[2025-02-03 23:52:35] INFO:     127.0.0.1:60460 - "GET /v1/batches/batch_6ea3847b-0585-43c0-9689-918d656a2bf4 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url="http://127.0.0.1:30020/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-02-03 23:52:38] INFO:     127.0.0.1:37266 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-03 23:52:38] INFO:     127.0.0.1:37266 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-03 23:52:38 TP0] Prefill batch. #new-seq: 6, #new-token: 6, #cached-token: 324, cache hit rate: 48.27%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-03 23:52:38 TP0] Prefill batch. #new-seq: 152, #new-token: 1834, #cached-token: 6526, cache hit rate: 65.50%, token usage: 0.01, #running-req: 6, #queue-req: 0
[2025-02-03 23:52:38 TP0] Prefill batch. #new-seq: 274, #new-token: 8192, #cached-token: 6850, cache hit rate: 55.32%, token usage: 0.01, #running-req: 158, #queue-req: 68


[2025-02-03 23:52:38 TP0] Prefill batch. #new-seq: 69, #new-token: 2068, #cached-token: 1700, cache hit rate: 54.17%, token usage: 0.03, #running-req: 431, #queue-req: 1


[2025-02-03 23:52:39 TP0] Decode batch. #running-req: 500, #token: 35025, token usage: 0.08, gen throughput (token/s): 906.85, #queue-req: 0


[2025-02-03 23:52:40 TP0] Decode batch. #running-req: 500, #token: 55025, token usage: 0.12, gen throughput (token/s): 26194.43, #queue-req: 0


...

[2025-02-03 23:52:46 TP0] Decode batch. #running-req: 500, #token: 195025, token usage: 0.44, gen throughput (token/s): 19122.64, #queue-req: 0


[2025-02-03 23:52:47 TP0] Decode batch. #running-req: 500, #token: 215025, token usage: 0.48, gen throughput (token/s): 18303.78, #queue-req: 0


[2025-02-03 23:52:48] INFO:     127.0.0.1:42156 - "POST /v1/batches/batch_9e611d56-e273-4e8b-938f-cbfefe337249/cancel HTTP/1.1" 200 OK


[2025-02-03 23:52:51] INFO:     127.0.0.1:42156 - "GET /v1/batches/batch_9e611d56-e273-4e8b-938f-cbfefe337249 HTTP/1.1" 200 OK


[2025-02-03 23:52:51] INFO:     127.0.0.1:42156 - "DELETE /v1/files/backend_input_file-7405212d-5a89-414e-93ab-16e92090be2a HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)